# **Final Exam - Deep Network Development**



# **Exam Information**

---

- **Name:** Mir Mohibullah Sazid
- **Neptun ID:** GVFPV8
- **Date:** *03/01/2025*
- **Duration:** *9:00 AM – 11:00 AM*
- *Please fill in your details above before starting the exam.*



## **General Rules**

This notebook contains the task to be completed in order to pass the exam and the course. Below are the details:
1. **Implementing a network architecture**, including its **forward pass** function.
2. Additional **optional requirements** for bonus points towards final grade.
3. You have **2 hours** to complete the exam.
4. You may distribute the time as you see fit between the required and optional parts.
5. You are allowed to use any resource including: the internet, AI tools, practice notebooks, and more.
6. **It is strictly prohibited to use any form of communication** (e.g., Teams, WhatsApp, Messenger, etc.). **Violation will result in an immediate FAIL** of the exam.

---

### **Submission Guidelines**
- Submit your solution as a **`.ipynb` file** on **Canvas**.
- To **PASS**, your solution must:
  1. **Satisfy the minimum requirements** (i.e., a working implementation of the network architecture and forward pass).
  2. Be **submitted on time**.
  3. Be prepared to **orally defend your code** after submission.

---

### **Exam Retake Policy**
- If you **FAIL**, you are allowed to do **one retake**.  
- If you **FAIL AGAIN**, sadly, you **fail the course**.  

---

### **Grading**
- If you **PASS**, your final grade will be the **weighted average** of your assignment defenses (theory and code).

---

Good luck, and ensure you follow all the rules!


## **Requirements**

---

### **Minimum Requirements – Sufficient to Pass the Exam**
1. **Implement the layers of the architecture:**  
   Complete the architecture outlined in Section 1 by filling in the missing parts.
2. **Implement the forward function:**  
   Ensure the input and output of the forward function are correctly implemented.  
   
   **Note:** To meet these requirements, your final output must match the expected output.

---

### **Extra Requirements – For Grade Improvement and AI Lab Access**

---

3. **Text-to-Image with Image-Guided Embeddings:**  
- The goal is to perform text-to-image generation using an existing image as a guide for editing. The input text specifies modifications to the existing image, preserving its content while applying specific changes as described by the text.

   ➡️ **Reward: +1 to final grade**

---

4. **Replacing Text Encoder with Transformer:**  
- Replace the text encoder with a Transformer model.
- Test the new architecture to ensure it performs text-to-image editing correctly, by satisfying the expect output condition.

   ➡️ **Reward: Access to AI Lab**

---

Make sure to carefully follow the instructions provided in each cell to meet the requirements!


## **1. Required: Task Description**

Your task is to implement a custom neural network architecture along with its forward pass function.

This task is inspired by **text-to-image generation**, where a neural network maps a sequence of tokens representing textual information into a high-dimensional image. The text input is typically **tokenized** into a sequence of integers. This representation can be passed through an **encoder-decoder** architecture to generate images.

For this task, you will work with a simplified text-to-image representation in the form of a random tensor with the shape **(1, 10)**:
- The 1 indicates that there is a single input sample.
- 10 corresponds to the sequence length of the input text tokens.

Your implemented model will:
- Take this text token tensor as input.
- Encode it into a latent representation.
- Decode the latent representation to produce an output **image tensor of shape (1, 3, 256, 256)**, where:
    - 1 represents the batch size.
    - 3 indicates the RGB color channels of the image.
    - 256 × 256 corresponds to the height and width of the output image.

The primary objective is to correctly implement the neural network architecture and its forward pass to achieve the desired functionality.

To better view the architecture diagram:  
- **Right-click the image** and select **"Open image in a new tab"** to enable zoom for a clearer view.  
- Alternatively, you can **download the image** using the link below:  
  [Download Architecture Diagram](https://drive.google.com/file/d/1osVNVcsNGo-d9DCGVH1hJDw2nw4rMToR/view?usp=sharing)

---

### Diagram Preview:
![Architecture Diagram](https://drive.google.com/uc?export=view&id=1osVNVcsNGo-d9DCGVH1hJDw2nw4rMToR)


Necessary Imports and Data Loading

In [1]:
# Cell 0.1
import torch
import torch.nn as nn
import numpy as np

In [2]:
# Cell 0.2 (GPU is not needed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

### Input

In [3]:
# Cell 0.3 -> INPUT (DO NOT EDIT THIS CELL!)
vocab_size = 10
input_tokens = torch.randint(0, vocab_size, (1, 10))
print(input_tokens.shape)

torch.Size([1, 10])


### Architecture

In [4]:
import torch
import torch.nn as nn

class TextEncoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(TextEncoder, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_size = 128
        self.num_layers = 2

        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # LSTM layer
        self.lstm = nn.LSTM(input_size=embedding_dim, 
                            hidden_size=self.hidden_size, 
                            num_layers=self.num_layers, 
                            batch_first=True)

        # Linear layer
        self.linear = nn.Linear(self.hidden_size, 10 * 32 * 32)

    def forward(self, input_tokens):
        print("Text input:", input_tokens.shape)

        # Embedding
        embedded = self.embedding(input_tokens)
        print("After embedding:", embedded.shape)

        # LSTM
        lstm_out, _ = self.lstm(embedded)
        print("After LSTM:", lstm_out.shape)

        # Extract the output of the last time step
        last_hidden_state = lstm_out[:, -1, :]
        print("Last hidden state:", last_hidden_state.shape)

        # Linear transformation
        linear_out = self.linear(last_hidden_state)
        print("After linear layer:", linear_out.shape)

        # Reshape to (1, 10, 32, 32)
        out = linear_out.view(-1, 10, 32, 32)
        print("After reshape:", out.shape)

        return out

# Initialize and test the model
vocab_size = 10
embedding_dim = 256
input_tokens = torch.randint(0, vocab_size, (1, 10))

model = TextEncoder(vocab_size, embedding_dim)
output = model(input_tokens)
print("Model output shape:", output.shape)


Text input: torch.Size([1, 10])
After embedding: torch.Size([1, 10, 256])
After LSTM: torch.Size([1, 10, 128])
Last hidden state: torch.Size([1, 128])
After linear layer: torch.Size([1, 10240])
After reshape: torch.Size([1, 10, 32, 32])
Model output shape: torch.Size([1, 10, 32, 32])


In [13]:
import torch
import torch.nn as nn

class ImageDecoder(nn.Module):
    def __init__(self):
        super(ImageDecoder, self).__init__()
        
        # First ConvTranspose2D layer
        self.conv_trans1 = nn.ConvTranspose2d(in_channels=10, out_channels=16, kernel_size=12,stride=4,padding=0)
        
        
        
        # Conv2D + ReLU
        self.conv1 = nn.Conv2d(in_channels=10, out_channels=32, kernel_size=3, stride=1, padding=0)
        
        
        # MaxPool2D
        self.max_pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.relu1 = nn.ReLU()
        # BatchNorm2D
        self.batch_norm1 = nn.BatchNorm2d(num_features=32)

        self.max_pool1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        # Second Conv2D + ReLU
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=0)
        self.relu2 = nn.ReLU()

        self.batch_norm2 = nn.BatchNorm2d(num_features=32)
        
        # Conv2D (adds residual connection)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=0)
        
        # ConvTranspose2D
        self.conv_trans2 = nn.ConvTranspose2d(in_channels=64, out_channels=32, kernel_size=3, stride=1, padding=0)
        
        # Add operation (for skip connection)
        self.add1 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=8, stride=3, padding=0)
        
        # Final ConvTranspose2D layers
        self.final_conv_trans = nn.ConvTranspose2d(in_channels=32, out_channels=16, kernel_size=24, stride=19, padding=1)
        self.final_conv_trans2 = nn.ConvTranspose2d(in_channels=32, out_channels=8, kernel_size=3, stride=2, padding=1)
        self.final_conv = nn.Conv2d(in_channels=8, out_channels=3, kernel_size=16, stride=1, padding=0)

    def forward(self, text_embedding):
        print("Text embedding:", text_embedding.shape)
        
        # ConvTranspose2D
        x1 = self.conv_trans1(text_embedding)
        print("After first ConvTranspose2D:", x1.shape)

        x2 = self.conv1(text_embedding)
        print("After first Conv2D:", x2.shape)
        x = self.max_pool(x2)
        print("After MaxPool2D:", x.shape)
        x = self.relu1(x)
        print("After ReLU:", x.shape)
        x = self.batch_norm1(x)
        print("After BatchNorm2D:", x.shape)
        y5 = self.max_pool1(x)
        print("After MaxPool2D:", y5.shape)

        x3 = self.conv2(x2)
        print("After second Conv2D:", x3.shape)
        x = self.relu2(x3)
        print("After ReLU:", x.shape)
        y = self.batch_norm2(x)
        print("After BatchNorm2D:", y.shape)

        x4 = self.conv3(x3)
        print("After third Conv2D:", x4.shape)
        y1 = self.conv_trans2(x4)
        print("After second ConvTranspose2D:", y1.shape)

        y3 = y1 + y
        print("After add operation:", y3.shape)
        y4 = self.add1(y3)
        print("After add operation:", y4.shape)
        y6 = y4 + y5
        print("After add operation:", y6.shape)

        # Final ConvTranspose2D layers
        x = self.final_conv_trans(y6)
        print("After final ConvTranspose2D:", x.shape)

        x = torch.concat((x, x1), 1)
        print("After concatenation:", x.shape)

        x = self.final_conv_trans2(x)
        print("After final ConvTranspose2D:", x.shape)

        x = self.final_conv(x)
        print("After final Conv2D:", x.shape)
        
      
        
        return x

# Test the ImageDecoder with a dummy input
dummy_input = torch.randn(1, 10, 32, 32)  # Text embedding output
decoder = ImageDecoder()
output_image = decoder(dummy_input)
print("Output image shape:", output_image.shape)


Text embedding: torch.Size([1, 10, 32, 32])
After first ConvTranspose2D: torch.Size([1, 16, 136, 136])
After first Conv2D: torch.Size([1, 32, 30, 30])
After MaxPool2D: torch.Size([1, 32, 15, 15])
After ReLU: torch.Size([1, 32, 15, 15])
After BatchNorm2D: torch.Size([1, 32, 15, 15])
After MaxPool2D: torch.Size([1, 32, 7, 7])
After second Conv2D: torch.Size([1, 32, 28, 28])
After ReLU: torch.Size([1, 32, 28, 28])
After BatchNorm2D: torch.Size([1, 32, 28, 28])
After third Conv2D: torch.Size([1, 64, 26, 26])
After second ConvTranspose2D: torch.Size([1, 32, 28, 28])
After add operation: torch.Size([1, 32, 28, 28])
After add operation: torch.Size([1, 32, 7, 7])
After add operation: torch.Size([1, 32, 7, 7])
After final ConvTranspose2D: torch.Size([1, 16, 136, 136])
After concatenation: torch.Size([1, 32, 136, 136])
After final ConvTranspose2D: torch.Size([1, 8, 271, 271])
After final Conv2D: torch.Size([1, 3, 256, 256])
Output image shape: torch.Size([1, 3, 256, 256])


In [14]:
import torch
import torch.nn as nn

class TextEncoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(TextEncoder, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_size = 128
        self.num_layers = 2

        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # LSTM layer
        self.lstm = nn.LSTM(input_size=embedding_dim, 
                            hidden_size=self.hidden_size, 
                            num_layers=self.num_layers, 
                            batch_first=True)

        # Linear layer
        self.linear = nn.Linear(self.hidden_size, 10 * 32 * 32)

    def forward(self, input_tokens):
        print("Text input:", input_tokens.shape)

        # Embedding
        embedded = self.embedding(input_tokens)
        print("After embedding:", embedded.shape)

        # LSTM
        lstm_out, _ = self.lstm(embedded)
        print("After LSTM:", lstm_out.shape)

        # Extract the output of the last time step
        last_hidden_state = lstm_out[:, -1, :]
        print("Last hidden state:", last_hidden_state.shape)

        # Linear transformation
        linear_out = self.linear(last_hidden_state)
        print("After linear layer:", linear_out.shape)

        # Reshape to (1, 10, 32, 32)
        out = linear_out.view(-1, 10, 32, 32)
        print("After reshape:", out.shape)

        return out

In [15]:
class ImageDecoder(nn.Module):
    def __init__(self):
        super(ImageDecoder, self).__init__()
        
        # First ConvTranspose2D layer
        self.conv_trans1 = nn.ConvTranspose2d(in_channels=10, out_channels=16, kernel_size=12,stride=4,padding=0)
        
        
        
        # Conv2D + ReLU
        self.conv1 = nn.Conv2d(in_channels=10, out_channels=32, kernel_size=3, stride=1, padding=0)
        
        
        # MaxPool2D
        self.max_pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.relu1 = nn.ReLU()
        # BatchNorm2D
        self.batch_norm1 = nn.BatchNorm2d(num_features=32)

        self.max_pool1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        # Second Conv2D + ReLU
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=0)
        self.relu2 = nn.ReLU()

        self.batch_norm2 = nn.BatchNorm2d(num_features=32)
        
        # Conv2D (adds residual connection)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=0)
        
        # ConvTranspose2D
        self.conv_trans2 = nn.ConvTranspose2d(in_channels=64, out_channels=32, kernel_size=3, stride=1, padding=0)
        
        # Add operation (for skip connection)
        self.add1 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=8, stride=3, padding=0)
        
        # Final ConvTranspose2D layers
        self.final_conv_trans = nn.ConvTranspose2d(in_channels=32, out_channels=16, kernel_size=24, stride=19, padding=1)
        self.final_conv_trans2 = nn.ConvTranspose2d(in_channels=32, out_channels=8, kernel_size=3, stride=2, padding=1)
        self.final_conv = nn.Conv2d(in_channels=8, out_channels=3, kernel_size=16, stride=1, padding=0)

    def forward(self, text_embedding):
        print("Text embedding:", text_embedding.shape)
        
        # ConvTranspose2D
        x1 = self.conv_trans1(text_embedding)
        print("After first ConvTranspose2D:", x1.shape)

        x2 = self.conv1(text_embedding)
        print("After first Conv2D:", x2.shape)
        x = self.max_pool(x2)
        print("After MaxPool2D:", x.shape)
        x = self.relu1(x)
        print("After ReLU:", x.shape)
        x = self.batch_norm1(x)
        print("After BatchNorm2D:", x.shape)
        y5 = self.max_pool1(x)
        print("After MaxPool2D:", y5.shape)

        x3 = self.conv2(x2)
        print("After second Conv2D:", x3.shape)
        x = self.relu2(x3)
        print("After ReLU:", x.shape)
        y = self.batch_norm2(x)
        print("After BatchNorm2D:", y.shape)

        x4 = self.conv3(x3)
        print("After third Conv2D:", x4.shape)
        y1 = self.conv_trans2(x4)
        print("After second ConvTranspose2D:", y1.shape)

        y3 = y1 + y
        print("After add operation:", y3.shape)
        y4 = self.add1(y3)
        print("After add operation:", y4.shape)
        y6 = y4 + y5
        print("After add operation:", y6.shape)

        # Final ConvTranspose2D layers
        x = self.final_conv_trans(y6)
        print("After final ConvTranspose2D:", x.shape)

        x = torch.concat((x, x1), 1)
        print("After concatenation:", x.shape)

        x = self.final_conv_trans2(x)
        print("After final ConvTranspose2D:", x.shape)

        x = self.final_conv(x)
        print("After final Conv2D:", x.shape)
        
      
        
        return x

#### Test your implementation

In [16]:
#DO NOT MODIFY THIS CELL

embedding_dim = 256
print("Encoder:")
text_encoder = TextEncoder(vocab_size, embedding_dim)
text_embedding = text_encoder(input_tokens)
print("Decoder:")
image_decoder = ImageDecoder()
image = image_decoder(text_embedding)

try:
    assert text_embedding.shape == (1, 10, 32, 32), "Encoded text shape is incorrect."
    assert image.shape == (1, 3, 256, 256), "Decoded image shape is incorrect."
    print("\n🎉 Congratulations! Your implementation is correct. You passed the minimum requirement! 🎉")
except AssertionError as e:
    print(f"\n❌ Error: {e}")

Encoder:
Text input: torch.Size([1, 10])
After embedding: torch.Size([1, 10, 256])
After LSTM: torch.Size([1, 10, 128])
Last hidden state: torch.Size([1, 128])
After linear layer: torch.Size([1, 10240])
After reshape: torch.Size([1, 10, 32, 32])
Decoder:
Text embedding: torch.Size([1, 10, 32, 32])
After first ConvTranspose2D: torch.Size([1, 16, 136, 136])
After first Conv2D: torch.Size([1, 32, 30, 30])
After MaxPool2D: torch.Size([1, 32, 15, 15])
After ReLU: torch.Size([1, 32, 15, 15])
After BatchNorm2D: torch.Size([1, 32, 15, 15])
After MaxPool2D: torch.Size([1, 32, 7, 7])
After second Conv2D: torch.Size([1, 32, 28, 28])
After ReLU: torch.Size([1, 32, 28, 28])
After BatchNorm2D: torch.Size([1, 32, 28, 28])
After third Conv2D: torch.Size([1, 64, 26, 26])
After second ConvTranspose2D: torch.Size([1, 32, 28, 28])
After add operation: torch.Size([1, 32, 28, 28])
After add operation: torch.Size([1, 32, 7, 7])
After add operation: torch.Size([1, 32, 7, 7])
After final ConvTranspose2D: torch

## **2. Optional: +1 to the Final Grade**
- Add another input tensor: a random tensor of size (1, 3, 256, 256).
- Implement an Image Encoder with a few layers to encode the tensor. The encoding process should follow the example image provided.
- Combine the encoded image embeddings with the text embeddings using cross-attention, following the example image provided.

You should only add these new parts and reuse the ImageDecoder previously created. The final output should still be the same as in the previously required task (1,3,256,256).

To better view the architecture diagram:  
- **Right-click the image** and select **"Open image in a new tab"** to enable zoom for a clearer view.  
- Alternatively, you can **download the image** using the link below:  
  [Download Architecture Diagram](https://drive.google.com/file/d/1nIgqhyPq0eKWEvT7leqoa0ZeBApCCs6u/view?usp=sharing)

---

### Diagram Preview:
![Architecture Diagram](https://drive.google.com/uc?export=view&id=1nIgqhyPq0eKWEvT7leqoa0ZeBApCCs6u)


#### New Input - create a random tensor of size (1,3,256,256)

In [ ]:
# ADD YOUR CODE HERE

#### Image Encoder - create the image encoder, following the example provided.

In [ ]:
class ImageEncoder(nn.Module):
    def __init__(self):
        super(ImageEncoder, self).__init__()
        # ADD YOUR CODE HERE

    def forward(self, new_image):
        print("New image:", new_image.shape)
        # ADD YOUR CODE HERE
        return out

#### Combine with Cross-Attention

In [ ]:
class CrossAttention(nn.Module):
    def __init__(self, embed_dim=1024):
        super(CrossAttention, self).__init__()
        # ADD YOUR CODE HERE

    def forward(self, text_embedding, image_embedding):
        # ADD YOUR CODE HERE
        return out

#### Test your implementation

In [ ]:
# DO NOT MODIFY THIS CELL
image_encoder = ImageEncoder()
image_embedding = image_encoder(new_image)

embed_dim = 1024
cross_attention = CrossAttention(embed_dim=embed_dim)
combine = cross_attention(text_embedding, image_embedding)

image = image_decoder(combine)

try:
    assert image_embedding.shape == (1, 10, 32, 32), "Encoded image shape is incorrect."
    assert combine.shape == (1, 10, 32, 32), "Combined cross attention shape is incorrect."
    assert image.shape == (1, 3, 256, 256), "Combined cross attention shape is incorrect."
    print("\n🎉 Congratulations! Your implementation is correct. You increased your final grade by 1! 🎉")
except AssertionError as e:
    print(f"\n❌ Error: {e}")

## **3. Optional: Access to AI Lab**
- Replace the text encoder with a Transformer model. This involves:
    - Maximizing the sequence length to 16.
    - Using BertTokenizer.
    - Adding Positional Encoding.
    - Defining a Transformer Encoder.


To better view the architecture diagram:  
- **Right-click the image** and select **"Open image in a new tab"** to enable zoom for a clearer view.  
- Alternatively, you can **download the image** using the link below:  
  [Download Architecture Diagram](https://drive.google.com/file/d/1bwrPryFGAAFF3OoJ3Z7UUzpVJaYvijUg/view?usp=sharing)

---

### Diagram Preview:
![Architecture Diagram](https://drive.google.com/uc?export=view&id=1bwrPryFGAAFF3OoJ3Z7UUzpVJaYvijUg)


In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, embed_dim, max_len=16):
        super(PositionalEncoding, self).__init__()
        # ADD YOUR CODE HERE

    def forward(self, x):
        # ADD YOUR CODE HERE

        return x

In [ ]:
from transformers import BertTokenizer

# Transformer Encoder
class TransformerTextEncoder(nn.Module):
    def __init__(self, embed_dim, max_len=16, num_heads=4, num_layers=2):
        super(TransformerTextEncoder, self).__init__()
        # ADD YOUR CODE HERE

    def forward(self, input_text):
        # ADD YOUR CODE HERE

        return out

In [ ]:

#DO NOT MODIFY THIS CELL
new_image = torch.randn((1,3,256,256))
embedding_dim = 1024 # 32x32
print("Encoder:")
text_encoder = TransformerTextEncoder(embedding_dim)
input_text = ["Generate an image based on this text"]
text_embedding = text_encoder(input_text)

image_encoder = ImageEncoder()
image_embedding = image_encoder(new_image)

embed_dim = 1024
cross_attention = CrossAttention(embed_dim=embed_dim)
combine = cross_attention(text_embedding, image_embedding)

print("Decoder:")
image_decoder = ImageDecoder()
image = image_decoder(combine)

try:
    assert text_embedding.shape == (1, 16, 32, 32), "Encoded text shape is incorrect."
    assert image_embedding.shape == (1, 16, 32, 32), "Encoded image shape is incorrect."
    assert combine.shape == (1, 16, 32, 32), "Combined cross attention shape is incorrect."
    assert image.shape == (1, 3, 256, 256), "Decoded image shape is incorrect."
    print("\n🎉 Congratulations! Your implementation is correct. You passed the requirement for the AI Lab! 🎉")
except AssertionError as e:
    print(f"\n❌ Error: {e}")